In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer
from nltk import word_tokenize, sent_tokenize
from nltk.util import ngrams, bigrams , trigrams
from nltk.corpus import stopwords
from nltk import ne_chunk

In [ ]:
data = pd.read_csv(r"../input/disaster-dataset/train.csv")
data1= pd.read_csv(r"../input/disaster-dataset/test.csv")
sub = pd.read_csv(r"../input/disaster-dataset/sample_submission.csv")

In [ ]:
data

In [ ]:
import missingno as msg
msg.matrix(data)

In [ ]:
import missingno as msg
msg.matrix(data1)

In [ ]:
data.isnull().count()

In [ ]:
data

In [ ]:
data= data.drop(['id'], axis = 1)
data1= data1.drop(['id'], axis = 1)

In [ ]:
data.head()

In [ ]:
data['keyword'].value_counts().sort_values(ascending = False)

In [ ]:
sns.countplot(data.keyword)

In [ ]:
data.keyword.unique()

In [ ]:
data = data.drop( ['keyword'] , axis = 1)
data1 = data1.drop(['keyword'] , axis = 1)

In [ ]:
data

In [ ]:
data['location'].value_counts().sort_values(ascending = False)

In [ ]:
data.location.unique()

In [ ]:
data= data.drop(['location'],axis =1)
data1= data1.drop(['location'],axis=1)

In [ ]:
data

In [ ]:
import re
def clean_data(tweet):
    tweet = re.sub("RT @[\w]*:", "", tweet)
    tweet = re.sub("@[\w]*", "", tweet)
    tweet = re.sub("https://[A-Za-z0-9./]", "", tweet)
    tweet = re.sub("\n", "", tweet)
    tweet = re.sub("&amp", "", tweet)
    tweet = re.sub("#", "", tweet)
    tweet = re.sub(r"[^\w]", ' ', tweet )
    return tweet

In [ ]:
data

In [ ]:
data['text'] = data['text'].apply(lambda x: clean_data(x))
data1['text'] = data1['text'].apply(lambda x: clean_data(x))

In [ ]:
data

In [ ]:
data['text'] = data['text'].apply(lambda x:x.lower())
data1['text'] = data1['text'].apply(lambda x:x.lower())

In [ ]:
data

In [ ]:
stop = stopwords.words('english')
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
data

In [ ]:
l=data1.text.values
l

In [ ]:
stop = stopwords.words('english')
data1['text'] = data1['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
data1

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
fake_data = data[data["target"] == 0]
all_words = ' '.join([text for text in fake_data.text])
wordcloud = WordCloud(width= 800, height= 500,
                          max_font_size = 110,
                          collocations = False).generate(all_words)
plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
from wordcloud import WordCloud
real_data = data[data["target"] == 1]
all_words = ' '.join([text for text in real_data.text])
wordcloud = WordCloud(width= 800, height= 500,
                          max_font_size = 110,
                          collocations = False).generate(all_words)
plt.figure(figsize=(10,7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
cvt = TfidfVectorizer(tokenizer=word_tokenize , ngram_range=(1,2))

In [ ]:
x,y = data.text , data.target


In [ ]:
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size= 0.3 , random_state =100)

In [ ]:
cvt.fit(x_train)
x_train_trans = cvt.transform(x_train)

x_test_trans = cvt.transform(x_test)

In [ ]:
x_train_trans

In [ ]:
x_test_trans

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [ ]:
model.fit(x_train_trans , y_train)

In [ ]:
pred = model.predict(x_test_trans)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(pred , y_test)